<a href="https://colab.research.google.com/github/CrazyTiger8903/Paper-Review/blob/main/Boundary-Unlearning/boundary_unlearning_rapid_forgetting_of_deep_networks_via_shifting_the_decision_boundary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# boundary unlearning: rapid forgetting of deep networks via shifting the decision boundary(CVPR 2023)

## 1.Introduction

---

- Machine Unlearing : 머신러닝 모델이 일부 학습 데이터를 잊도록 하거나 그 계통을 삭제시키는 것을 의미

- 왜 필요한가?
  1.   "the right to be forgotten" : 개인 데이터의 삭제를 요청하면 기업은 반드시 삭제해야한다(GDPR)
  2.   머신러닝 모델에 유용
    -   데이터 중독 공격(학습 데이터를 조작하여 모델의 성능 저하 또는 잘못 판단하도록 하는 공격)
    -   시간이 지나 유효하지 않은 학습 데이터
    -   학습 후 학습 데이터가 오류로 판단

- 초기연구
  1.   처음 부터 retraining -> 시간 오래 걸림
  2.   모델 파라미터를 scrubbing(정리)하여 forgetting data(삭제 대상 데이터)의 영향을 제거 -> 파라미터 space 차원이 너무 크기 때문에 cost가 비싸다.

- Retrained Model의 결정 공간 특징
  1.   forgetting samples가 모델의 결정 공간 전체에 퍼져 있다. 즉 forgetting samples의 결정 경계가 깨졌다.
  2.   대부분의 forgetting samples는 다른 클러스터의 경계로 이동
    - closest-to-boundary criterion : 결정 공간의 클러스터 경계에 있는 샘플은 예측 시 높은 불확실성을 보일 가능성이 크다.
  3. 이러한 특징은 machine unlearning의 2가지 목표에 자연스럽게 부합한다.
    - Utility guarantee(유용성 보장) : forgetting data(삭제 대상 데이터)에 대해서는 일반화 성능이 낮아야 하며, 나머지 데이터에 대한 예측 성능은 유지되어야 한다.
    - privacy guarantee : unlearned 모델은 삭제 대상 데이터의 정보를 유출해서는 안된다.
    - 이 논문에서는 forgetting data의 경계만 파괴하고, 나머지는 유지함으로써 Utility guarantee를 달성하고, forgetting data를 다른 클러스터의 경계로 밀어냄으로써 privacy guarantee를 달성한다.

  <img src='https://drive.google.com/uc?id=1KuoVnQL0p0YejkocQIq172TBEVomaasD'/>

- Boundary Unlearning
  1. 결정 공간에서 결정 경계를 이동시켜 빠르고 효과적으로 모델에서 삭제 대상 클래스를 잊게 하는 방법인 "Boundary Unlearning" 제안
  2. Boundary Shrink : forgetting class(삭제 대상 클래스)를 다른 클래스로 분할하여 결정 경계를 파괴.
  3. Boundary Expanding : forgetting 데이터를 shadow class로 매핑 후 이 클래스를 제거하여 활성화를 분산.




## 2. Related Work

---



## 3. Preliminaries and Notation
---
1. $D$ : training dataset. 전체 학습 데이터셋으로, 입력데이터 x와 클래스 레이블 y로 구성

2. $D_{f} $ : Forgetting Data. 삭제해야 할 데이터의 부분 집합. 주로 특정 클래스의 모든 샘플로 구성됨. (예를 들면, dog 클래스 전체를 삭제.)

3. $D_{r} $ :  Remaining Training Data. $D_{f} $(삭제 대상 데이터)를 제외한 나머지 데이터. 즉 유지하려는 정보들

4. $f_{ w_{0} } $ : 학습데이터 D에 대해 학습된 Original 모델. w0로 파라미터화 되어 있다. $f_{ w_{0} }(x) $는 학습된 모델이 x에 대해 출력하는 로짓을 나타낸다.

5. $f_{w*} $ : retrained 모델($D_{r} $로 다시 학습된 모델) 최적의 데이터 삭제 모델로 간주.

6. $f_{w'}$ : 데이터 삭제를 수행한 후 얻어진 모델.

- 최종 목표 : fw'(데이터 삭제 후 얻어진 모델)이 fw*(재학습 시킨 모델)와 최대한 유사하게 만드는 것.

7. decision boundary(결정경계) : 입력 x가 클래스 i와 j 중 어느 클래스로 분류될지 결정하는 경계. 두 클래스의 확률이 동일해지는 지점

## 4. Proposed Methods
---
### 4.1. Boundary Shrink
- Random Labeling
  - 랜덤하게 라벨링된 forgetting 데이터를 사용해 파인튜닝을 진행하는 방법
  - 가장 직관적인 방법이지만, 나머지 클래스의 경계도 랜덤하게 이동 시켜 나머지 데이터에 대한 모델 utility저하

- Retrained 모델의 결정 경계에서 확인할 수 있듯, 대부분의 forgetting 샘플은 랜덤한 클래스가 아닌 특정 클래스들로 예측. 즉, 특징 공간에서 forgetting 샘플과 다른 클래스의 샘플들 간의 유사성 발견이 중요하다.

- 제안
  - 적대적 공격(adversarial attacks)을 기반으로 이웃 탐색 방법을 제안
    - 가장 가까운 결정 경계를 넘어 적대적 예제(adversarial examples)를 생성할 수 있는 방법
    - forgetting 샘플의 결정 경계를 이동시키는 방향을 알려주며, 원래 모델에서 cross samples(잘못된 예측)를 예측함으로써 가장 가까우면서도 잘못된 라벨을 얻을 수 있다.
  - cross samples의 라벨을 해당하는 forgetting 샘플에 할당
  - 재할당 된 모든 샘플로 원래 모델을 파인튜닝하면, forgetting 클래스의 경계가 정확한 방향으로 축소될 수 있다.

1. Original 모델 학습
  - $ w_0 = \arg\min_w \sum_{(x_i, y_i) \in D} L(x_i, y_i, w) $
  - $w_0$(original 모델의 가중치)
  - D(전체 데이터세트)에 대해 손실 L을 최소화하는 최적의 파라미터를 나타냄
2. 이웃검색방법을 사용하여 가장 가까우면서 잘못된 레이블 찾기
  - $x'_f = x_f + \epsilon \cdot \text{sign} \left( \nabla_{x_f} L(x_f, y, w_0) \right)$
  - original 모델의 손실 함수의 $ x_{f}$에 대한 gradient 계산
  - sign을 통해 gradient의 부호를 취해 방향을 결정. 즉 크기는 제외하고 방향성 만을 추출한다.
  - gradient의 방향성에 일정한 크기의 노이즈 크기 $\epsilon$(엡실론)을 추가
  - 이를 통해 $ x_{f}$의 결정 경계를 넘도록 노이즈를 추가한다. - 적대적 공격과 유사(PGD Attack)
  - $y_{nbi} \leftarrow \text{softmax} \left( f_{w_0}(x'_f) \right)$
  - $x'_f$(cross sample)에 대해 original 모델 $f_{w_0}$를 사용해 가장 가깝지만 잘못된 레이블 $y_{nbi}$를 예측
3. fine-tunning
  - $w' = \arg \min_w \sum_{(x_i, y_{nbi}) \in D_f} L(x_i, y_{nbi}, w_0)$
  - forgetting 클래스의 boundary shrink를 위해 재할당된 샘플로 original 모델을 파인 튜닝. 이를 통해 최적의 파라미터 w'을 획득

- 이를 통해 모델이 학습한 데이터 간의 관계를 최대한 보존하면서 결정 경계를 조정할 수 있게 된다.

### 4.2. Boundary Expanding



## Performance Evaluation

---
1. Utility Guarantee
  - 모델 정확도를 통해 평가
  - 다른 베이스라인 모델과 비교하였을때, Boundary Shrink는 Df에 대한 정확도를 크게 줄이면서, Dr에 대한 정확도를 최소한으로 감소시킨다.
  - Boundary Expanding은 Df에 대한 정확도가 조금 더 높지만, 성능과 시간 사이의 균형을 맞춘 빠른 대안으로 사용 가능하다.

  <img src='https://drive.google.com/uc?id=1VOQwfeJkygY4VMXLg5UvJ7YypsJ966Yc'/>

2. Privacy Guarantee
  - ASR(Attack Success Rate)를 통해 평가
  - ASR이 100%에 가까울수록 Df에 대한 정보가 덜 제거되었음을 의미
  - ASR이 0%에 가까울수록 Streisand 효과를 나타낼 수 있음(Df의 모든 샘플이 하나의 클래스로 예측되어, 삭제 데이터에 대한 더 많은 정보를 제공할 가능성을 의미)
  - Finetune의 경우 높은 Utility 성능을 보였지만, Privacy 성능의 경우 낮은 성능을 보임(삭제 데이터에 대한 정보를 거의 제거하지 못했음)
  - Boundary Shrink와 Boundary Expanding 방법 모두 준수한 성능을 보임

   <img src='https://drive.google.com/uc?id=18rlJDnCK9U9uAnrgLPMa3uH-JXBSaXjE'/>

3. Computational Complexity
  - Retrain 대비 많은 시간 단축
  - Boundary Shrink는 교차 샘플 생성(cross-sample generation) 과정 때문에 Boundary Expanding보다 약간 더 많은 시간이 소요

  <img src='https://drive.google.com/uc?id=1JCzMtLRl9GYfdGpXYOa5T-aSkCt0pRck'/>


4. Attention Map
  - Retrain 모델의 attention map을 통해 unlearning되었을 때 모델이 얼굴 영역에 집중하지 않는 다는 것을 확인할 수 있다.
  - Boundary Shrink로 unlearning된 경우 모델의 주위를 오직 배경에만 집중시키도록 한 것을 확인
  - Boundary Expanding의 경우 주위를 완전히 배경으로 전환시키지 못했지만, 여전히 얼굴 외부 영역으로 집중을 전환시킨 것을 확인

  <img src='https://drive.google.com/uc?id=1C5Nz__kfF9uu9UrdQkiw9N0v6OarI9ng'/>

5. Visualization of Decision Space
  - Boundary Shrink를 적용한 후 가장 가까운 클래스로 예측되는 것을 확인
  - Retrain 모델처럼 클러스터가 완전히 퍼지지는 않았지만, forgetting data의 결정 공간이 근처의 클래스에 의해 나눠지는 것을 확인할 수 있다.
  - Boundary Expanding의 경우 forgetting data의 클러스터가 중심에서 밀려나는 것을 확인할 수 있다.

  <img src='https://drive.google.com/uc?id=1M5hdFb0NjhRjBIjf21NGTrmitm3haGDe'/>

6. Distribution of the Entropy of Model Output
  - 엔트로피가 낮음 : 데이터를 예측할 때 더 확신을 갖고 예측함을 의미
  - Retrained Model은 Df의 엔트로피 수치가 크게 증가하는 것을 확인할 수 있다.
  - Boundary Shrink와 Expanding의 결과 역시 Retrained의 결과와 유사한 것을 확인할 수 있다.
  - Random Labels의 경우 Df의 엔트로피 수치가 상당히 큰 것을 볼 수 있는데, 이는 Retrained model과 큰 차이를 보인다.

  <img src='https://drive.google.com/uc?id=1VnyEitYD_dFq9ZcuTtdVGT00PB5aQ1U1'/>

## Conclusion

---
- 결정 경계를 이동시켜 학습된 DNN에서 특정 클래스 전체의 정보를 제거하는 최초의 머신 언러닝 기법인 Boundary Unlearning을 제안
- 과도한 계산 자원을 소모하지 않으며, 기존 학습 파이프라인에 개입하지 않음
- Utility와 Privacy 보장 모두에서 빠르고 효율적인 망각 성능을 보임


# 코드 구현

## 1. Original & Retrain 모델 만들기
1-1. pytorch 라이브러리를 이용하여 ResNet18 모델을 구현하고 CIFAR-10 데이터셋을 학습한 'Original'모델을 만들고 정확도 등을 평가하세요.

1-2. 10개 클래스 중 한 개의 클래스를 선택해 이를 제외한 나머지 9개 클래스의 데이터로 학습한 'Retrain'모델을 만들고 각 데이터에 대한 정확도 등을 평가하세요 (train forgetset, train remainset, test forgetset, test remainset...)


- 1-1 Result
  - Train Remainset Accuracy (others): 99.99%
  - Train Forgetset Accuracy (deer): 100.00%
  - Test Remainset Accuracy (others): 83.08%
  - Test Forgetset Accuracy (deer): 82.50%

    <img src='https://drive.google.com/uc?id=1I5Eud6OMcq1G7I6mbq4lIRl3TM_py7lH' width='450' height='450' />

- 1-2 Result
  - Train Remainset Accuracy (others): 99.99%
  - Train Forgetset Accuracy (deer): 0.00%
  - Test Remainset Accuracy (others): 83.93%
  - Test Forgetset Accuracy (deer): 0.00%

    <img src='https://drive.google.com/uc?id=19dLiQm8hgs5sPy3JRgKkIlfSs53xHXQZ' width='450' height='450' />

In [ ]:
"""
boundary unlearning: rapid forgetting of deep networks via shifting the decision boundary(CVPR 2023)

Original 모델(Cifar10 데이터세트로 훈련된 모델) 학습 및 시각화 코드
Train Remainset Accuracy (others): 99.99%
Train Forgetset Accuracy (deer): 100.00%
Test Remainset Accuracy (others): 83.08%
Test Forgetset Accuracy (deer): 82.50%

to do : 모듈화
"""
import torch
from torchvision import datasets, transforms
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import random
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

save_dir = "./Result"
os.makedirs(save_dir, exist_ok=True)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 데이터셋 분리 함수
def split_datasets(dataset, target_class):
    forget_indices = [i for i, (_, label) in enumerate(dataset) if label == target_class]
    remain_indices = [i for i, (_, label) in enumerate(dataset) if label != target_class]
    return Subset(dataset, forget_indices), Subset(dataset, remain_indices)

# 삭제 대상 클래스 - deer(4)
target_class = 4

train_forgetset, train_remainset = split_datasets(train_dataset, target_class)
test_forgetset, test_remainset = split_datasets(test_dataset, target_class)

train_forget_loader = DataLoader(train_forgetset, batch_size=64, shuffle=True)
train_remain_loader = DataLoader(train_remainset, batch_size=64, shuffle=True)
test_forget_loader = DataLoader(test_forgetset, batch_size=64, shuffle=False)
test_remain_loader = DataLoader(test_remainset, batch_size=64, shuffle=False)

# ResNet18 모델 초기화
def create_model(num_classes=10):
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

# 학습 함수
def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# 평가 함수
def evaluate_model(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# Original 모델 학습
best_accuracy = 0.0
epoch = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
original_model = create_model(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(original_model.parameters(), lr=0.001, momentum=0.9)

# 학습
for i in range(epoch):
    train_model(original_model, train_loader, criterion, optimizer, device)
    accuracy = evaluate_model(original_model, test_loader, device)
    print(f"Epoch {i+1}, Test Accuracy: {accuracy * 100:.2f}%")

    # 최고 정확도 갱신 시
    if accuracy > best_accuracy:
        best_accuracy = accuracy

        # 최고 성능 모델 가중치 저장
        best_weight_path = os.path.join(save_dir, "Org_Model.pth")
        torch.save(original_model.state_dict(), best_weight_path)
        print(f"New best accuracy: {accuracy * 100:.2f}% - Weights saved at {best_weight_path}")


print("Training complete. Best weights saved as 'best_model.pth'")


# TSNE 시각화 함수
def plot_tsne(model, remain_loader, forget_loader, device, title, classes, save_path):
    model.eval()  # 모델을 평가 모드로 전환
    features = []  # 추출된 특징 저장 리스트
    predictions = []  # 예측 클래스 저장 리스트
    dataset_type = []  # "Remain" 또는 "Forget" 구분 저장 리스트

    # Remainset 데이터에서 특징 및 예측 결과 추출
    with torch.no_grad():
        for inputs, _ in remain_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            features.append(outputs.cpu().numpy())  # 모델 출력 특징 저장
            predictions.append(torch.argmax(outputs, dim=1).cpu().numpy())  # 예측 결과 저장
            dataset_type.extend(["Remain"] * inputs.size(0))  # Remainset 데이터 구분 추가

    # Forgetset 데이터에서 특징 및 예측 결과 추출
    with torch.no_grad():
        for inputs, _ in forget_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            features.append(outputs.cpu().numpy())  # 모델 출력 특징 저장
            predictions.append(torch.argmax(outputs, dim=1).cpu().numpy())  # 예측 결과 저장
            dataset_type.extend(["Forget"] * inputs.size(0))  # Forgetset 데이터 구분 추가

    # 데이터를 합침
    features = np.concatenate(features, axis=0)
    predictions = np.concatenate(predictions, axis=0)
    dataset_type = np.array(dataset_type)

    # TSNE로 차원 축소
    # tsne = TSNE(n_components=2, perplexity=40, learning_rate=300, n_iter=3000, random_state=42)
    tsne = TSNE(n_components=2, perplexity=50, learning_rate=500, n_iter=4000, random_state=42)
    reduced_features = tsne.fit_transform(features)

    # 시각화
    plt.figure(figsize=(12, 10))
    colors = plt.cm.Paired(np.linspace(0, 1, len(classes)))

    # Remainset 데이터 시각화
    remain_features = reduced_features[dataset_type == "Remain"]
    remain_predictions = predictions[dataset_type == "Remain"]
    for i, class_name in enumerate(classes):
        indices_remain = remain_predictions == i
        plt.scatter(
            remain_features[indices_remain, 0],
            remain_features[indices_remain, 1],
            label=f"Class {i+1} ({class_name})",
            alpha=0.5,
            s=30,
            color=colors[i]
        )

    # Forgetset 데이터 시각화
    forget_features = reduced_features[dataset_type == "Forget"]
    forget_predictions = predictions[dataset_type == "Forget"]
    for i, class_name in enumerate(classes):
        indices_forget = forget_predictions == i
        plt.scatter(
            forget_features[indices_forget, 0],
            forget_features[indices_forget, 1],
            alpha=0.6,
            s=30,
            edgecolor="black",
            color=colors[i]
        )

    plt.title(title, fontsize=20, fontweight="bold")
    plt.legend(loc="center left", bbox_to_anchor=(1, 0.5), fontsize="small", markerscale=1.0)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.tight_layout()

    # 시각화 이미지 저장
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"t-SNE visualization saved at {save_path}")

best_weight_path = os.path.join(save_dir, "Org_Model.pth")

final_model = create_model(num_classes=10).to(device)
final_model.load_state_dict(torch.load(best_weight_path))
final_model.eval()
print(f"Loaded best weights from {best_weight_path}")

# 각 데이터셋에서 정확도 평가
train_forget_acc = evaluate_model(final_model, train_forget_loader, device)
train_remain_acc = evaluate_model(final_model, train_remain_loader, device)
test_forget_acc = evaluate_model(final_model, test_forget_loader, device)
test_remain_acc = evaluate_model(final_model, test_remain_loader, device)

# 결과 출력
print(f"Train Remainset Accuracy (others): {train_remain_acc * 100:.2f}%")
print(f"Train Forgetset Accuracy (deer): {train_forget_acc * 100:.2f}%")
print(f"Test Remainset Accuracy (others): {test_remain_acc * 100:.2f}%")
print(f"Test Forgetset Accuracy (deer): {test_forget_acc * 100:.2f}%")

# TSNE 시각화
tsne_predictions_path = os.path.join(save_dir, "tsne_Org_Model.png")
print("\nGenerating TSNE visualization...")

plot_tsne(
    final_model,
    test_remain_loader,
    test_forget_loader,
    device,
    "TSNE Visualization(Original Model)",
    classes,
    tsne_predictions_path
)

In [ ]:
"""
boundary unlearning: rapid forgetting of deep networks via shifting the decision boundary(CVPR 2023)

Retrain 모델(Dr, 즉 forgetting data를 제외한 데이터세트로 학습한 모델) 학습 및 시각화 코드
Train Remainset Accuracy (others): 99.99%
Train Forgetset Accuracy (deer): 0.00%
Test Remainset Accuracy (others): 83.93%
Test Forgetset Accuracy (deer): 0.00%

to do : 모듈화
"""
import torch
from torchvision import datasets, transforms
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import random
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

save_dir = "./Result"
os.makedirs(save_dir, exist_ok=True)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 데이터셋 분리 함수
def split_datasets(dataset, target_class):
    forget_indices = [i for i, (_, label) in enumerate(dataset) if label == target_class]
    remain_indices = [i for i, (_, label) in enumerate(dataset) if label != target_class]
    return Subset(dataset, forget_indices), Subset(dataset, remain_indices)

# 삭제 대상 클래스 - deer(4)
target_class = 4

train_forgetset, train_remainset = split_datasets(train_dataset, target_class)
test_forgetset, test_remainset = split_datasets(test_dataset, target_class)

train_forget_loader = DataLoader(train_forgetset, batch_size=64, shuffle=True)
train_remain_loader = DataLoader(train_remainset, batch_size=64, shuffle=True)
test_forget_loader = DataLoader(test_forgetset, batch_size=64, shuffle=False)
test_remain_loader = DataLoader(test_remainset, batch_size=64, shuffle=False)

# ResNet18 모델
def create_model(num_classes=10):
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

# 학습 함수
def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# 평가 함수
def evaluate_model(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

best_accuracy = 0.0
epoch = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
retrain_model = create_model(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(retrain_model.parameters(), lr=0.001, momentum=0.9)

# 학습
for i in range(epoch):
    train_model(retrain_model, train_remain_loader, criterion, optimizer, device)
    accuracy = evaluate_model(retrain_model, test_remain_loader, device)
    print(f"Epoch {i+1}, Test Accuracy: {accuracy * 100:.2f}%")

    # 최고 성능 저장
    if accuracy > best_accuracy:
        best_accuracy = accuracy

        best_weight_path = os.path.join(save_dir, "retrain_model.pth")
        torch.save(retrain_model.state_dict(), best_weight_path)
        print(f"New best accuracy: {accuracy * 100:.2f}% - Weights saved at {best_weight_path}")

final_model = create_model(num_classes=10).to(device)
final_model.load_state_dict(torch.load(best_weight_path))
final_model.eval()
print(f"Loaded best weights from {best_weight_path}")

train_forget_acc = evaluate_model(final_model, train_forget_loader, device)
train_remain_acc = evaluate_model(final_model, train_remain_loader, device)
test_forget_acc = evaluate_model(final_model, test_forget_loader, device)
test_remain_acc = evaluate_model(final_model, test_remain_loader, device)

print(f"Train Remainset Accuracy (others): {train_remain_acc * 100:.2f}%")
print(f"Train Forgetset Accuracy (deer): {train_forget_acc * 100:.2f}%")
print(f"Test Remainset Accuracy (others): {test_remain_acc * 100:.2f}%")
print(f"Test Forgetset Accuracy (deer): {test_forget_acc * 100:.2f}%")

## 2. Baseline 만들기
2-1. 앞서 학습한 Original 모델을 가지고 논문 baseline으로 설명된 'Random Labels' 기법에 대해 Machine Unlearning을 수행하고 정확도 등을 평가하세요. (Random Labels 기법은 forgetting data에 임의의 라벨을 부여하여 fine-tune합니다.)

[2-1 Result]
- Train Remainset Accuracy (others): 89.39%
- Train Forgetset Accuracy (deer): 8.88%
- Test Remainset Accuracy (others): 73.58%
- Test Forgetset Accuracy (deer): 6.70%

Selective forgetting of deep networks at a finer level than samples(CoRR 2020)

[랜덤 라벨 디스틸레이션(Random Label Distillation, RLD)]
특정 입력 데이터가 네트워크에 무작위 라벨을 학습하도록 강제하여 기존 학습된 정보를 "망각" 시킴

[학습 목표]
Df(잊어야할 데이터)에 대한 성능은 떨어뜨리고, Dr(나머지 데이터)의 성능은 유지하는 것이 목표
이를 위해 2가지의 손실 함수를 결합하여 사용

### 망각 손실(Lf)
- Df 데이터에 대한 성능을 낮추기 위해 사용
- 랜덤 라벨 디스틸레이션(Random Label Distillation, RLD)을 사용하여 Df를 망각 시킴
- $ L_{\text{RLD}}(f_\theta, x_f) = L_{\text{cls}}(f_\theta(x_f), u) $
  - $f_\theta(x_f)$ : 모델이 xf에 대해 출력한 로짓 값
  - u : xf에 대해 무작위로 선택된 클래스
  - $L_{\text{cls}}$ : Softmax Cross-Entropy Loss
  - 즉, xf를 원래의 정답 라벨이 아닌 랜덤 라벨로 학습시킴으로써, 해당 데이터에 대한 기존 학습 내용을 잊게 한다.

### 기억 손실(R)
- Dr 데이터에 대한 성능을 유지하기 위해 사용
- Elastic Weight Consolidation(EWC) 방식을 사용하여 모델이 기존 데이터의 중요한 특징을 유지하도록 정규화
- $ R(f_\theta, f_{\theta_{\text{old}}}) = (\theta - \theta_{\text{old}})^T F(\theta_{\text{old}})(\theta - \theta_{\text{old}}) $
  - $\theta$ : 현재 학습 중인 모델의 파라미터 벡터
  - $\theta_{\text{old}}$ : 과거 모델(original 모델)의 파라미터 벡터
  - $F(\theta_{\text{old}})$ : Fisher 정보 행렬. 각 파라미터의 중요도를 나타냄
-  Fisher 정보 행렬($F(\theta)_{ii}$)
  - 파라미터 𝜃에 대한 손실 함수의 민감도(각 파라미터가 손실에 얼마나 중요한지에 대한 정보)
  - $F(\theta_{\text{old}})_{ii} = |D|^{-1} \sum_{(x, l) \in D} \left[ \frac{\partial L_{\text{cls}}(f_\theta(x), l)}{\partial \theta_i} \right]^2$
  - 모든 파라미터 $\theta_i$에 대해 손실함수 $L_{\text{cls}}$의 편미분의 제곱의 평균의 대각 성분
- $ \frac{\partial R}{\partial \theta_i} = 2 \cdot F(\theta_{\text{old}})_{ii} \cdot (\theta_i - \theta_{\text{old}, i})$
  - EWC 손실 항에 대해 $\theta_i$에 대한 기울기
  - 즉, Fisher 정보가 클수록 기울기의 크기가 커진다.
  - 경사하강법에서 큰 기울기는 파라미터 업데이트를 강하게 유도하여 손실을 빠르게 줄인다.
  - 따라서 중요한 파라미터(Fisher 정보가 큰)는 원래 값에 머무르려는 경향이 강해지고, 큰 변화가 제한된다.
- <확인해보기> 이차형식(Quadratic Forms)...????

### 최종 손실 함수
- $L = L_f(f_\theta, D_f) + \lambda_{\text{KL}} \cdot R(f_\theta, f_{\theta_{\text{old}}})$
  - $\lambda_{\text{KL}}$ : R에 대한 가중치(0 이상, 고정값)



In [ ]:
"""
boundary unlearning: rapid forgetting of deep networks via shifting the decision boundary(CVPR 2023)

Random Labels 모델(랜덤으로 라벨링한 forgetting data를 사용해 original 모델을 파인튜닝) 학습 및 시각화 코드

Train Remainset Accuracy (others): 89.39%
Train Forgetset Accuracy (deer): 8.88%
Test Remainset Accuracy (others): 73.58%
Test Forgetset Accuracy (deer): 6.70%

to do :
- 모듈화
- ASR 추가
"""

"""
Selective forgetting of deep networks at a finer level than samples(CoRR 2020)

[랜덤 라벨 디스틸레이션(Random Label Distillation, RLD)]
특정 입력 데이터가 네트워크에 무작위 라벨을 학습하도록 강제하여 기존 학습된 정보를 "망각" 시킴

[학습 목표]
Df(잊어야할 데이터)에 대한 성능은 떨어뜨리고, Dr(나머지 데이터)의 성능은 유지하는 것이 목표
이를 위해 2가지의 손실 함수를 결합하여 사용

1. 망각 손실(Lf)
Df 데이터에 대한 성능을 낮추기 위해 사용
랜덤 라벨 디스틸레이션(Random Label Distillation, RLD)을 사용하여 Df를 망각 시킴

2. 기억 손실(R)
Dr 데이터에 대한 성능을 유지하기 위해 사용
Elastic Weight Consolidation(EWC) 방식을 사용하여 모델이 기존 데이터의 중요한 특징을 유지하도록 정규화
"""

import torch
from torchvision import datasets, transforms
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import random
import numpy as np
import os
import copy
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# 랜덤 시드 고정
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

save_dir = "./Result"
os.makedirs(save_dir, exist_ok=True)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 데이터셋 분리 함수
def split_datasets(dataset, target_class):
    forget_indices = [i for i, (_, label) in enumerate(dataset) if label == target_class]
    remain_indices = [i for i, (_, label) in enumerate(dataset) if label != target_class]
    return Subset(dataset, forget_indices), Subset(dataset, remain_indices)

target_class = 4  # deer

train_forgetset, train_remainset = split_datasets(train_dataset, target_class)
test_forgetset, test_remainset = split_datasets(test_dataset, target_class)

train_forget_loader = DataLoader(train_forgetset, batch_size=64, shuffle=True)
train_remain_loader = DataLoader(train_remainset, batch_size=64, shuffle=True)
test_forget_loader = DataLoader(test_forgetset, batch_size=64, shuffle=False)
test_remain_loader = DataLoader(test_remainset, batch_size=64, shuffle=False)

def create_model(num_classes=10):
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

# 랜덤 라벨 생성 함수
def generate_random_labels(labels, num_classes):
    random_labels = torch.randint(0, num_classes, labels.size(), device=labels.device)
    while (random_labels == labels).any():
        random_labels = torch.randint(0, num_classes, labels.size(), device=labels.device)
    return random_labels

# Random Label Distillation(RLD) Loss : CrossEntropyLoss 사용
# 망각 손실(Lf) 참고
def random_label_distillation_loss(outputs, random_labels):
    criterion = nn.CrossEntropyLoss()
    return criterion(outputs, random_labels)

# Elastic Weight Consolidation (EWC) Loss
# 기억 손실(R) 참고
def ewc_loss(model, fisher_matrix, old_weights, lambda_ewc):
    ewc_loss_value = 0
    for name, param in model.named_parameters():
        if name in fisher_matrix:
            fisher = fisher_matrix[name]   # fisher 정보(파라미터 중요도)
            old_weight = old_weights[name] # original 모델의 파라미터 값
            # (현재 파라미터 - 과거 파라미터)^2에 Fisher 정보로 가중치 부여 후 합산
            ewc_loss_value += (fisher * (param - old_weight).pow(2)).sum()
    # EWC 손실에 가중치 부여
    return lambda_ewc * ewc_loss_value

# Fisher 정보 계산 함수
# original 모델로부터 파라미터 중요도 계산
def calculate_fisher_information(model, dataloader, device):
    # fisher행렬 초기화(모델의 각 파라미터에 대해 0으로 초기화)
    fisher_matrix = {name: torch.zeros_like(param) for name, param in model.named_parameters() if param.requires_grad}
    model.eval()

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        # fθ(x)
        outputs = model(inputs)
        # Lcls(fθ(x),l)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        # ∂Lcls(fθ(x),l) / ∂θi
        loss.backward()

        # 최종 fisher 정보 행렬
        # 손실 함수의 기울기 제곱의 평균으로 정의
        for name, param in model.named_parameters():
            if param.requires_grad and param.grad is not None:
                fisher_matrix[name] += param.grad.pow(2) / len(dataloader)

    return fisher_matrix

# 학습 함수
def train_model(model, dataloader, optimizer, fisher_matrix, old_weights, lambda_ewc, device):
    model.train()
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # 랜덤 라벨 생성
        random_labels = generate_random_labels(labels, len(classes))
        optimizer.zero_grad()
        outputs = model(inputs)

        # 망각 손실 (RLD)
        rld_loss = random_label_distillation_loss(outputs, random_labels)

        # 기억 손실 (EWC)
        ewc_loss_value = ewc_loss(model, fisher_matrix, old_weights, lambda_ewc)

        # 총 손실
        loss = rld_loss + ewc_loss_value
        loss.backward()
        optimizer.step()


# 평가 함수
def evaluate_model(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# 모델 초기화 및 학습
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Random_Labels_Model = create_model(num_classes=len(classes)).to(device)
original_model = create_model(num_classes=len(classes)).to(device)

original_weights_path = "./Result/Org_Model.pth"
original_model.load_state_dict(torch.load(original_weights_path))
print(f"Loaded original model weights from {original_weights_path}")

Random_Labels_Model.load_state_dict(copy.deepcopy(original_model.state_dict()))

# original_model 가중치
old_weights = {name: param.clone().detach() for name, param in original_model.named_parameters()}
# 전체 데이터에 대해 Fisher 정보 행렬 계산
fisher_matrix = calculate_fisher_information(original_model, train_loader, device)

# We set λKL = 10^5, lr = 10^−5 throughout experiments.
optimizer = optim.SGD(Random_Labels_Model.parameters(), lr=1e-5)
lambda_ewc = 1e5
best_tradeoff_score = -float('inf')

for epoch in range(10):
    print(f"Epoch {epoch + 1}/10")
    train_model(Random_Labels_Model, train_forget_loader, optimizer, fisher_matrix, old_weights, lambda_ewc, device)

    #train_remain_acc = evaluate_model(Random_Labels_Model, train_remain_loader, device)
    test_remain_acc = evaluate_model(Random_Labels_Model, test_remain_loader, device)
    #train_forget_acc = evaluate_model(Random_Labels_Model, train_forget_loader, device)
    test_forget_acc = evaluate_model(Random_Labels_Model, test_forget_loader, device)

    # print(f"Train Remainset Accuracy: {train_remain_acc * 100:.2f}%")
    # print(f"Train Forgetset Accuracy: {train_forget_acc * 100:.2f}%")
    # print(f"Test Remainset Accuracy: {test_remain_acc * 100:.2f}%")
    # print(f"Test Forgetset Accuracy: {test_forget_acc * 100:.2f}%")

    tradeoff_score = test_remain_acc - test_forget_acc

    if tradeoff_score > best_tradeoff_score:
        best_tradeoff_score = tradeoff_score
        best_weight_path = os.path.join(save_dir, "Random_Labels_Model.pth")
        torch.save(Random_Labels_Model.state_dict(), best_weight_path)
        print(f"New best trade-off score: {tradeoff_score:.4f} - Model saved to {best_weight_path}")

print("Training complete.")

final_model = create_model(num_classes=10).to(device)
final_model.load_state_dict(torch.load(best_weight_path))
final_model.eval()
print(f"Loaded best weights from {best_weight_path}")

train_forget_acc = evaluate_model(final_model, train_forget_loader, device)
train_remain_acc = evaluate_model(final_model, train_remain_loader, device)
test_forget_acc = evaluate_model(final_model, test_forget_loader, device)
test_remain_acc = evaluate_model(final_model, test_remain_loader, device)

# 결과 출력
print(f"Train Remainset Accuracy (others): {train_remain_acc * 100:.2f}%")
print(f"Train Forgetset Accuracy (deer): {train_forget_acc * 100:.2f}%")
print(f"Test Remainset Accuracy (others): {test_remain_acc * 100:.2f}%")
print(f"Test Forgetset Accuracy (deer): {test_forget_acc * 100:.2f}%")

## 3. 논문의 아이디어 구현하기
3-1. 논문의 메인 아이디어 중 하나인 'Boundary Shrink'를 구현하고 각 데이터에 대한 정확도 등을 평가하세요. 이때 adversarial attack 기법인 PGD attack를 직접 구현해보세요 (어려우시면 다른데서 찾아오셔도 됩니다.)

3-2. Retrain, Random Labels, Boundary Shrink 세 기법의 결과를 서로 비교하고 평가하세요.

[3-1 Result]
- Train Remainset Accuracy (others): 94.02%
- Train Forgetset Accuracy (deer): 3.22%
- Test Remainset Accuracy (others): 77.32%
- Test Forgetset Accuracy (deer): 2.50%

    <img src='https://drive.google.com/uc?id=1NL9QlqVnLxRq0cpxJz81Z6dLjfkf8b6y' width='450' height='450' />

[3-2 Result]
- Retrain 모델
  - Forgetset의 Accuracy가 0%로, 삭제된 데이터에 대한 정보가 완전히 제거됨. 하지만 처음부터 다시 학습해야 하므로, 연산 비용과 시간이 매우 큼.
- Random Labels
  - Boundary Shrink보다 적은 계산 비용으로 Forgetset의 Accuracy를 낮출 수 있다. 하지만, Boundary Shrink에 비해 성능이 낮다.
- Boundary Shrink
  - Forgetset Accuracy가 2.50%로, Random Labels보다 강력한 Privacy Guarantee를 제공한다.
  - 또한  Remaining Class의 성능 손상(Train Remainset Accuracy 94.02%)을 최소화 하였다.


<br>

  | **Metric**                          | **Original** | **Retrain** | **Random Labels** | **Boundary Shrink** |
|-------------------------------------|--------------|-------------|--------------------|----------------------|
| **Train Remainset Accuracy (others)** | 99.99%       | 99.99%      | 89.39%            | 94.02%              |
| **Train Forgetset Accuracy (deer)**  | 100.00%      | 0.00%       | 8.88%             | 3.22%               |
| **Test Remainset Accuracy (others)** | 83.08%       | 83.93%      | 73.58%            | 77.32%              |
| **Test Forgetset Accuracy (deer)**   | 82.50%       | 0.00%       | 6.70%             | 2.50%               |

<br>

<img src='https://drive.google.com/uc?id=1Q31HKVlS-XTPBtHbs6orWKHlry7xSh6h' width='1000' height='450' />


In [ ]:
"""
boundary unlearning: rapid forgetting of deep networks via shifting the decision boundary(CVPR 2023)

Boundary Shrink 모델 학습 및 시각화(https://www.dropbox.com/scl/fi/j3hgtrvp1vjptk9qz5ck6/Boundary-Unlearning-Code.zip?rlkey=h32gro8ysi4umtolzmi54gbdo&e=1&dl=0)

Train Remainset Accuracy (others): 94.02%
Train Forgetset Accuracy (deer): 3.22%
Test Remainset Accuracy (others): 77.32%
Test Forgetset Accuracy (deer): 2.50%

to do :
- 모듈화
- ASR 추가
"""
import torch
from torchvision import datasets, transforms
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import random
import numpy as np
import os
import copy
from tqdm import tqdm
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import torch.distributions as distributions
import torch
from torch import nn
from tqdm import tqdm

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

def split_datasets(dataset, target_class):
    forget_indices = [i for i, (_, label) in enumerate(dataset) if label == target_class]
    remain_indices = [i for i, (_, label) in enumerate(dataset) if label != target_class]
    return Subset(dataset, forget_indices), Subset(dataset, remain_indices)

target_class = 4
train_forgetset, train_remainset = split_datasets(train_dataset, target_class)
test_forgetset, test_remainset = split_datasets(test_dataset, target_class)

train_forget_loader = DataLoader(train_forgetset, batch_size=64, shuffle=True)
train_remain_loader = DataLoader(train_remainset, batch_size=64, shuffle=True)
test_forget_loader = DataLoader(test_forgetset, batch_size=64, shuffle=False)
test_remain_loader = DataLoader(test_remainset, batch_size=64, shuffle=False)

def create_model(num_classes=10):
    model = resnet18(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

"""
adversarial attack이란? : perturbation(사람의 눈으로는 구별 불가능하지만, 모델의 예측에 영향을 주도록 하는 노이즈)를 추가하여 모델의 오분류를 유도하는 공격 기법
대표적인 기법으로는 FGSM과 PGD가 존재한다.

1. FGSM(Fast Gradient Sign Method)
    Explaining and Harnessing Adversarial Examples(ICLR 2015)
    오분류를 일으킬 수 있는 최적의 perturbation(노이즈, 델타)를 찾는 방법
    - 손실함수 L의 입력 x에 대한 gradient를 계산.
    - sign을 취하여 부호(방향)만을 추출
    - 일정한 값(엡실론)을 추가
    한번의 기울기 계산만 수행하므로 빠르고 계산 비용이 낮다.

2. PGD(Projected Gradient Descent)
    Towards Deep Learning Models Resistant to Adversarial Attacks(ICLR 2018)
    FGSM을 반복 수행하여 gradient를 업데이트
    각 과정에서  step size(=lr) 지정
    -> 더 강력한 adversarial sample을 생성 가능하다.
    각 과정에서 projection을 통해 perturbation(델타)을 일정 범위(L-infinity norm)내로 제한한다.
"""
class LinfPGD:
    def __init__(self, model=None, bound=None, step=None, iters=None, norm=False, random_start=False, device=None):
        """
        Args:
            model : 공격 대상 모델 - test 모델
            bound (float): 허용 범위 (L-infinity 기준 epsilon 값) - 0.1
            step (float): 한 번의 업데이트에서 변경할 크기 (step size) - 2 / 255
            iters (int): 적대적 샘플 생성을 반복할 횟수 - 5
            norm (bool): 입력 데이터를 정규화할지 여부 - True
            random_start (bool): 초기 샘플에 무작위 변동을 추가할지 여부 - True
            device
        """
        self.model = model
        self.bound = bound
        self.step = step
        self.iter = iters
        self.norm = norm
        if self.norm:
            # CIFAR10 데이터셋 기준 정규화 평균 및 표준편차
            self.mean = (0.4914, 0.4822, 0.2265)
            self.std = (0.2023, 0.1994, 0.2010)
        self.rand = random_start
        self.device = device
        # CrossEntropyLoss 사용
        self.criterion = nn.CrossEntropyLoss().to(device)

    def perturb(self, x, y, model=None, bound=None, step=None, iters=None, x_nat=None, device=None):
        """
        PGD(Projected Gradient Descent) attack을 수행하여 적대적 샘플 생성

        Args:
            x : 원본 입력 데이터
            y : 원본 입력 데이터의 정답 라벨
            model : 공격 대상 모델 - test 모델
            bound (float): 허용 범위 (L-infinity 기준 epsilon 값) - 0.1
            step (float): 한 번의 업데이트에서 변경할 크기 (step size) - 2 / 255
            iters (int): 적대적 샘플 생성을 반복할 횟수 - 5
            x_nat (torch.Tensor): 정규화되지 않은 원본 입력 데이터 - None
            device
        """
        criterion = self.criterion   # CrossEntropyLoss
        model = model or self.model
        bound = bound or self.bound
        step = step or self.step
        iters = iters or self.iter
        device = device or self.device

        # 모델 그레디언트 초기화
        model.zero_grad()

        # 정규화되지 않은 원본 데이터를 생성 (기본 설정).
        # torch.detach().clone() : 그래디언트 계산 그래프에서 분리 후 복사하여 새로운 메모리 공간에 저장
        if x_nat is None:
            x_nat = self.inverse_normalize(x.detach().clone().to(device))
        else:
            x_nat = self.inverse_normalize(x_nat.detach().clone().to(device))

        # 적대적 샘플 초기화
        x_adv = x.detach().clone().requires_grad_(True).to(device)

        # 적대적 샘플에 노이즈 추가한 상태로 시작
        if self.rand:
            # 허용 범위 내에서 랜덤한 노이즈를 생성
            rand_perturb_dist = distributions.uniform.Uniform(-bound, bound)
            rand_perturb = rand_perturb_dist.sample(sample_shape=x_adv.shape).to(device)
            # 생성한 노이즈 추가 및 범위 제한
            x_adv = self.clamper(self.inverse_normalize(x_adv) + rand_perturb, self.inverse_normalize(x_nat),
                                 bound=bound, inverse_normalized=True)
            # 노이즈 추가한 초기 x_adv 정규화 및 이산화
            x_adv = self.normalize(self.discretize(x_adv)).detach().clone().requires_grad_(True)

        # PGD attack 반복 -> 최적의 적대적 샘플 반환
        for i in range(iters):
            # 현재 적대적 샘플에 대한 모델(original 모델) 출력
            adv_pred = model(x_adv)
            # loss 계산. 모델 예측값과 실제 정답 간의 손실 계산
            loss = criterion(adv_pred, y)
            # 손실함수의 그래디언트 계산 - 수식(2)
            loss.backward()
            # 그래디언트의 부호 계산 - 수식(2)
            grad_sign = x_adv.grad.data.detach().sign()
            # x_adv를 업데이트: x_adv에 step size만큼 perturbation 추가 - 수식(2)
            x_adv = self.inverse_normalize(x_adv) + grad_sign * step
            # 범위 제한
            x_adv = self.clamper(x_adv, x_nat, bound=bound, inverse_normalized=True)
            model.zero_grad()

        # 최종적으로 적대적 샘플 반환(노이즈가 추가된 x)
        return x_adv.detach().to(device)

    # 입력 데이터 정규화
    def normalize(self, x):
        # x shape : (batch_size, channel, height, width)
        # norm이 True일 경우에는 정규화 후 반환
        # norm이 False일 경우에는 그대로 반환
        if self.norm:
            y = x.clone().to(x.device)
            # 각 체널별로 정규화 (x - 평균)/표준편차
            y[:, 0, :, :] = (y[:, 0, :, :] - self.mean[0]) / self.std[0]  # R체널
            y[:, 1, :, :] = (y[:, 1, :, :] - self.mean[1]) / self.std[1]  # G체널
            y[:, 2, :, :] = (y[:, 2, :, :] - self.mean[2]) / self.std[2]  # B체널
            return y
        return x

    # 입력 데이터 역정규화
    def inverse_normalize(self, x):
        if self.norm:
            y = x.clone().to(x.device)
            y[:, 0, :, :] = y[:, 0, :, :] * self.std[0] + self.mean[0]
            y[:, 1, :, :] = y[:, 1, :, :] * self.std[1] + self.mean[1]
            y[:, 2, :, :] = y[:, 2, :, :] * self.std[2] + self.mean[2]
            return y
        return x

    # 데이터 이산화 -> 계산 이득?
    def discretize(self, x):
        return torch.round(x * 255) / 255

    # 적대적 샘플(x_adv)이 원본 샘플(x_nat)의 허용 범위(bound)를 넘지 않도록 제한(clamp)하는 함수
    # torch.clamp(x, a, b) : x의 값을 a, b 범위 안으로 조정해주는 함수
    def clamper(self, x_adv, x_nat, bound=None, inverse_normalized=False):
        # 역정규화 안되어 있으면 역정규화 진행
        if not inverse_normalized:
            x_adv = self.inverse_normalize(x_adv)
            x_nat = self.inverse_normalize(x_nat)

        # 픽셀 값 차이를 bound로 제한
        clamp_delta = torch.clamp(x_adv - x_nat, -bound, bound)

        # 노이즈 더해주기
        x_adv = x_nat + clamp_delta
        # 범위 제한(0~1)사이로
        x_adv = torch.clamp(x_adv, 0., 1.)
        # 이산화(discretize) 및 정규화(normalize) 후 반환, 이후 추가 그래디언트 계산 가능하도록 requires_grad 설정
        return self.normalize(self.discretize(x_adv)).clone().detach().requires_grad_(True)


def inf_generator(iterable):
    iterator = iterable.__iter__()
    while True:
        try:
            yield iterator.__next__()
        except StopIteration:
            iterator = iterable.__iter__()

###################################################################################################################################################

# boundary_shrink
def boundary_shrink(ori_model, train_forget_loader, dt, dv, test_loader, device, evaluate,
                    bound=0.1, step=2 / 255, iter=5, poison_epoch=10, forget_class=0, path='./'):

    # 정규화, 무작위 시작
    norm = True
    random_start = True

    # test_model : 적대적 샘플 생성에 사용
    # unlearn_model : 학습(unlearning)에 사용
    test_model = copy.deepcopy(ori_model).to(device)
    unlearn_model = copy.deepcopy(ori_model).to(device)

    # LinfPGD 인스턴스 생성(적대적 샘플 생성을 위한)
    adv = LinfPGD(test_model, bound, step, iter, norm, random_start, device)

    # loss / 옵티마이저 설정
    # 논문 : For the fine-tune process in Boundary Unlearning we use a learning rate of 10−5 for 10 epochs.
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(unlearn_model.parameters(), lr=1e-5, momentum=0.9)

    # num_hits = 0
    # num_sum = 0
    best_tradeoff_score = -float('inf')
    best_weight_path = os.path.join(path, "shrink_model.pth")

    # boundary shrink 학습 과정
    for epoch in range(poison_epoch):
        print(f"Epoch {epoch + 1}/{poison_epoch}")

        for x, y in tqdm(train_forget_loader, desc=f"Epoch {epoch + 1} Batches"):
            x, y = x.to(device), y.to(device)

            # test모델(적대적 샘플 생성을 위한 모델)을 사용해 적대적 샘플을 생성
            # 적대적 샘플 : 노이즈 추가된 x
            test_model.eval()
            x_adv = adv.perturb(x, y, model=test_model, device=device)

            # 적대적 샘플에 대한 예측값 출력 및 예측 라벨 계산
            adv_logits = test_model(x_adv)
            pred_label = torch.argmax(adv_logits, dim=1)

            # num_hits += (y != pred_label).float().sum()
            # num_sum += y.shape[0]

            # unlearn모델 학습
            unlearn_model.train()
            optimizer.zero_grad()
            # unlearn모델에 적대적 샘플을 넣어 모델의 예측 출력
            ori_logits = unlearn_model(x)
            # loss 계산 - 모델의 예측값과 적대적 샘플의 라벨(바뀐 라벨) 손실 계산
            loss = criterion(ori_logits, pred_label)
            # 손실함수 그래디언트 계산 및 업데이트
            loss.backward()
            optimizer.step()

        test_remain_acc = evaluate_model(unlearn_model, test_remain_loader, device)
        test_forget_acc = evaluate_model(unlearn_model, test_forget_loader, device)

        print(f"Test Remainset Accuracy: {test_remain_acc * 100:.2f}%")
        print(f"Test Forgetset Accuracy: {test_forget_acc * 100:.2f}%")
        tradeoff_score = test_remain_acc - test_forget_acc

        if tradeoff_score > best_tradeoff_score:
            best_tradeoff_score = tradeoff_score
            torch.save(unlearn_model.state_dict(), best_weight_path)
            print(f"New best trade-off score: {tradeoff_score:.4f} - Model saved to {best_weight_path}")

    # asr = (num_hits / num_sum).float()
    # print('Attack Success Ratio (ASR):', asr)

    return unlearn_model

###################################################################################################################################################


# 정확도 평가 함수
def evaluate_model(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# 모델 생성 및 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ori_model = create_model(num_classes=10).to(device)
original_weights_path = "./Result/Org_Model.pth"
ori_model.load_state_dict(torch.load(original_weights_path))
print(f"Original model loaded from {original_weights_path}")

save_dir = "./Result"
os.makedirs(save_dir, exist_ok=True)
evaluation = ''

unlearn_model = boundary_shrink(ori_model, train_forget_loader, train_dataset, test_dataset,
                                                    test_loader, device, evaluation,
                                                    forget_class=target_class, path=save_dir)

# 디바이스 설정 및 모델 로드
best_weight_path = os.path.join(save_dir, "shrink_model.pth")
final_model = create_model(num_classes=10).to(device)
final_model.load_state_dict(torch.load(best_weight_path))

# 각 데이터셋에서 정확도 평가
train_forget_acc = evaluate_model(final_model, train_forget_loader, device)
train_remain_acc = evaluate_model(final_model, train_remain_loader, device)
test_forget_acc = evaluate_model(final_model, test_forget_loader, device)
test_remain_acc = evaluate_model(final_model, test_remain_loader, device)

# 결과 출력
print(f"Train Remainset Accuracy (others): {train_remain_acc * 100:.2f}%")
print(f"Train Forgetset Accuracy (deer): {train_forget_acc * 100:.2f}%")
print(f"Test Remainset Accuracy (others): {test_remain_acc * 100:.2f}%")
print(f"Test Forgetset Accuracy (deer): {test_forget_acc * 100:.2f}%")


## 4. 평가
4-1. 논문과 실험의 주요 contribution, weakness, 개선할 점, 구현 시 느낀점 등, 본인의 생각을 자유롭게 적어주세요

1. 주요 Contribution
  - 결정 경계를 이동시켜 특정 클래스 정보를 제거하는 최초의 unlearning 기법을 제안함
  - Utility와 Privacy 보장을 동시에 만족하면서도 빠르고 효율적인 망각 성능을 실험적으로 입증함

2. Weakness
  - Boundary Shrink는 적대적 샘플 생성 과정에서 높은 computational cost를 요구하며, 대규모 데이터세트나 복잡한 모델에서는 성능 저하 가능성이 있다고 생각한다.

3. 개선할 점
  - 대규모 데이터셋과 복잡한 모델에 대해서도 일관된 성능을 유지하는지 추가 검증이 필요하다고 생각한다.

4. 느낀점
  - 논문에서는 Remaining Class(Dr)의 Accuracy가 99%, 98% 수준으로 유지된다고 되어 있으나, 실험 결과는 90%, 94%로 다소 낮게 나왔다. 이는 실험 환경이나 구현 세부사항, 혹은 데이터 전처리 과정에서의 차이 때문일 가능성이 있어 추가적인 확인이 필요해 보인다.
  - 논문에서 제안된 수식(Random Labels의 망각 손실 함수와 Boundary Shrink의 PGD 공격)을 이해하고 구현하는 데 어려움을 느꼈다. 특히, 제안된 방법들이 개념적으로 이해는 되었으나, 이를 실제 코드로 구현하는 데 시간이 걸렸다.
  - 결론적으로, 논문을 깊이 이해하고 구현하는 과정에서 수학적 직관과 코딩 역량을 강화해야 한다는 필요성을 느꼈으며, 이를 위해 체계적인 학습과 실습이 필요하다고 느꼈다.